In [ ]:
#links to references
#https://scikit-learn.org/stable/user_guide.html
#https://colab.research.google.com/notebooks/intro.ipynb?utm_source=scs-index#scrollTo=-Rh3-Vt9Nev9

In [ ]:
#dataset
#https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

uploading train and test files

In [ ]:
from google.colab import files
train_uploaded = files.upload()


TypeError: ignored

In [ ]:
from google.colab import files
test_uploaded = files.upload()


In [ ]:
import pandas as pd

train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

uploaded files are not a dataframe yet

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py


In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
#checking all the missing values
missing_value=train_df.isnull().sum()
print(missing_value[missing_value>0])

In [ ]:
train_copy=train_df.drop(["MiscFeature"],axis=1)

In [ ]:
test_copy=test_df.drop(["MiscFeature"],axis=1)

In [ ]:
# lets drop features with a lot of missing values

In [ ]:
train_copy=train_copy.drop(["Alley","FireplaceQu","PoolQC", "Fence"],axis=1)
test_copy=test_copy.drop(["Alley","FireplaceQu","PoolQC", "Fence"],axis=1)


In [ ]:
#check and treat missing values and outliers for numerical and categorical variables
#separating categorical and numerical features

In [ ]:
numerical_traindata = train_copy.select_dtypes(include=[np.number])
categorical_traindata = train_copy.select_dtypes(exclude=[np.number])

numerical_testdata = test_copy.select_dtypes(include=[np.number])
categorical_testdata = test_copy.select_dtypes(exclude=[np.number])

In [ ]:
#for numerical features

In [ ]:
#checking all the missing values
missing_value=numerical_traindata.isnull().sum()
print(missing_value[missing_value>0])

In [ ]:

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
temp = imputer.fit_transform(numerical_traindata)

In [ ]:
temp

In [ ]:
#lost column headers so to get them back

In [ ]:
numerical_traindata = pd.DataFrame(temp, columns = numerical_traindata.columns)

In [ ]:
miss_temp=numerical_traindata.isnull().sum()

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3)
temp = imputer.fit_transform(numerical_testdata)

In [ ]:
numerical_testdata = pd.DataFrame(temp, columns = numerical_testdata.columns)

In [ ]:
numerical_testdata.head()

In [ ]:
#let's encode categorical data

In [ ]:
ordinal_features = ['LotShape','LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
               'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 
               'GarageQual', 'GarageCond' , 'Utilities']
nominal_features = [ 'MSZoning','Street', 'LotConfig', 'Neighborhood',
               'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
               'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'CentralAir', 'Electrical', 'Functional',
               'GarageType', 'GarageFinish', 'PavedDrive','SaleType', 'SaleCondition'
               , 'LandContour']

In [ ]:
categorical_traindata= categorical_traindata.astype('category')
categorical_testdata= categorical_testdata.astype('category')

In [ ]:
#label encoding for ordinal features
for col in ordinal_features:
    categorical_traindata[col] = categorical_traindata[col].cat.codes

In [ ]:
for col in ordinal_features:
    categorical_testdata[col] =categorical_testdata[col].cat.codes

In [ ]:

categorical_traindata.head()

In [ ]:
#onehot encoding for nominaldata


In [ ]:
categorical_traindata = pd.get_dummies(data = categorical_traindata, columns = nominal_features)

In [ ]:
categorical_testdata.head()

In [ ]:

categorical_testdata = pd.get_dummies(data = categorical_testdata, columns = nominal_features)

In [ ]:
categorical_testdata.head()

In [ ]:
#now club numerical and categorical data in both test andf train

In [ ]:
result_traindata = pd.concat([categorical_traindata, numerical_traindata], axis=1, join='inner')

In [ ]:
result_traindata.shape

In [ ]:
result_testdata = pd.concat([categorical_testdata, numerical_testdata], axis=1, join='inner')

In [ ]:
result_testdata.shape

In [ ]:
for column in result_traindata:
    if column not in result_testdata:
        result_traindata.drop([column], axis=1, inplace=True)



In [ ]:
for column in result_testdata:
    if column not in result_traindata:
        result_testdata.drop([column], axis=1, inplace=True)

In [ ]:
result_traindata #your whole train data

In [ ]:
 result_copy= result_traindata.copy()

In [ ]:
result_copy

In [ ]:
numerical_col_names =numerical_traindata.columns.values.tolist()

In [ ]:
type(numerical_col_names)

In [ ]:
numerical_col_names.remove('SalePrice')

In [ ]:
numerical_col_names


outlier treatment ---- Flooring

In [ ]:
#final result train data is in result copy

for column in numerical_col_names:
  first_quartile =np.percentile(result_copy[column], 25)
  third_quartile =  np.percentile(result_copy[column], 75)
#calculate the interquartilerange
  iqr = third_quartile - first_quartile
  # identify outliers
  cut_off = iqr * 1.5
  lower  = first_quartile - cut_off 
  upper  = third_quartile + cut_off
  result_copy[column]  = np.where(result_copy[column]>upper, upper,np.where(result_copy[column]<lower, lower,result_copy[column]))


In [ ]:
result_copy.shape

In [ ]:
#final result test data
#reomve outlier for test data
for column in numerical_col_names:
  first_quartile =np.percentile(result_copy[column], 25)
  third_quartile =  np.percentile(result_copy[column], 75)
#calculate the interquartilerange
  iqr = third_quartile - first_quartile
  # identify outliers
  cut_off = iqr * 1.5
  lower  = first_quartile - cut_off 
  upper  = third_quartile + cut_off
  result_testdata[column]  = np.where(result_testdata[column]>upper, upper,np.where(result_testdata[column]<lower, lower,result_testdata[column]))


In [ ]:
result_copy.head()

In [ ]:
import seaborn as sns
plt.subplot(211)
sns.boxplot(result_traindata['LotArea'])
plt.subplot(212)
sns.boxplot(result_copy['LotArea'])

In [ ]:
 target = train_df['SalePrice']

In [ ]:
# not doing normalization today

In [ ]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso, ElasticNet, BayesianRidge,RANSACRegressor,HuberRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.decomposition import PCA, KernelPCA
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, make_scorer
from sklearn.pipeline import Pipeline

In [ ]:
sc = ('Scaler', StandardScaler())
est =[]
est.append(('LinearRegression', Pipeline([sc, ('LinearRegression', LinearRegression())])))
est.append(('Ridge', Pipeline([sc, ('Ridge', Ridge())])))
est.append(('Lasso', Pipeline([sc, ('Lasso', Lasso())])))
est.append(('BayesianRidge', Pipeline([sc, ('BayesianRidge', BayesianRidge())])))
est.append(('ElasticNet', Pipeline([sc,('Elastic', ElasticNet())])))
est.append(('SGD', Pipeline([sc,('SGD', SGDRegressor())])))
est.append(('Huber', Pipeline([sc,('Huber', HuberRegressor())])))
est.append(('RANSAC', Pipeline([sc,('RANSAC', RANSACRegressor())])))
est.append(('GradientBoosting', Pipeline([sc,('GradientBoosting',GradientBoostingRegressor())])))
est.append(('AdaBoost', Pipeline([sc, ('AdaBoost', AdaBoostRegressor())])))
est.append(('ExtraTree', Pipeline([sc,('ExtraTrees', ExtraTreesRegressor())])))
est.append(('RandomForest', Pipeline([sc,('RandomForest', RandomForestRegressor())]))) 
est.append(('Bagging', Pipeline([sc,('Bagging', BaggingRegressor())])))
est.append(('KNeighbors', Pipeline([sc,('KNeighbors', KNeighborsRegressor())])))
est.append(('DecisionTree', Pipeline([sc,('DecisionTree', DecisionTreeRegressor())])))
est.append(('XGB', Pipeline([sc,('XGB', XGBRegressor())])))
#est.append('NeuralNet',Pipeline[sc,('NeuralNet',build_model())])


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
seed = 4
splits = 7
models_score ={}
for i in est:
    kfold = KFold(n_splits=splits, random_state=seed, shuffle=True)
    results = cross_val_score(i[1], result_copy, target, cv=kfold)
    models_score.update({i[0] : results.mean()})
    
sorted(models_score.items(), key= lambda v:v[1], reverse=True)

In [ ]:
#Scaling the train test dataset
sc=StandardScaler()
train_scaled_df = sc.fit_transform(result_traindata)
test_scaled_df = sc.fit_transform(result_testdata)

#choosing the model
best_model= ExtraTreesRegressor()
best_model.fit(train_scaled_df,target)

predictions = best_model.predict(test_scaled_df)

In [ ]:
predictions.shape

In [ ]:
submission = pd.DataFrame(columns=['Id', 'SalePrice'])
submission['Id'] = test_df['Id']
submission['SalePrice'] = predictions

submission.to_csv('submission.csv', index=False)

In [ ]:
submission.tail()

In [ ]:
from keras import models
from keras import layers

In [ ]:
def build_model():
      model = models.Sequential()
      model.add(layers.Dense(64,activation='relu',input_shape=(result_traindata.shape[1],)))
      model.add(layers.Dense(64,activation='relu'))
      model.add(layers.Dense(1))
      model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
      return model

In [ ]:
result_traindata.shape[1]

In [ ]:
nn_model = build_model()
nn_model.fit(train_scaled_df,target,epochs=500,batch_size=100,verbose=0)

In [ ]:
#nn_results =
import numpy as np

k = 4

num_val_samples = len (train_scaled_df) // k 
num_epochs = 500
all_mae_histories=[]
all_scores = []

for i in range (k):
  print('processing fold #', i) 
  val_data = train_scaled_df[i*num_val_samples: (i+1) * num_val_samples] 
  val_targets = target[i * num_val_samples: (i+1) * num_val_samples]

  partial_train_data = np.concatenate( [train_scaled_df[:i * num_val_samples], train_scaled_df[ (i+1) * num_val_samples:]],axis=0)

  partial_train_targets = np.concatenate ( [target[:i * num_val_samples],target[(i+1) * num_val_samples: ]],axis=0)

  model1=build_model()
  History=model1.fit(partial_train_data,partial_train_targets,validation_data=(val_data,val_targets), epochs=num_epochs, batch_size=1, verbose=0) 
  
  #val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
  mae_history=History.history['mean_absolute_error']
  all_mae_histories.append(mae_history)



In [ ]:
average_mae_history = [np.mean ([x[i] for x in all_mae_histories]) for i in range (num_epochs) ]

In [ ]:
import matplotlib.pyplot as plt

plt.plot (range (1, len (average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel("Validation MAE")
plt.show()

In [ ]:
all_scores

In [ ]:
import sklearn.metrics.SCORERS.keys

In [ ]:
test_nn=nn_model.predict(test_scaled_df)

In [ ]:
submission_nn = pd.DataFrame(columns=['Id', 'SalePrice'])
submission_nn['Id'] = test_df['Id']
submission_nn['SalePrice'] = test_nn

submission_nn.to_csv('submission_nn.csv', index=False)

In [ ]:
target.describe()

In [ ]:
import panda